In [2]:
import sys
print sys.path
import pandas as pd
import numpy as np
import outbrain.const as cst
import os

import os

reg = 10  # trying anokas idea of regularization

['', '/usr/local/lib/python2.7/site-packages/setuptools-20.2.2-py2.7.egg', '/usr/local/lib/python2.7/site-packages/distribute-0.6.14-py2.7.egg', '/usr/local/lib/python2.7/site-packages/pip-8.1.1-py2.7.egg', '/Users/dongjian/work/kaggle/outbrain/scripts', '/Users/dongjian/PycharmProjects/UserDetected', '/Users/dongjian/work/meituan', '/Users/dongjian/work/kaggle', '/Users/dongjian/work', '/Users/dongjian/work/packages/spark-1.5/python', '/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python27.zip', '/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7', '/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-darwin', '/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-mac', '/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-mac/lib-scriptpackages', '/usr/local/Cellar/python/2.7.11/Frameworks/Python.framewo

In [3]:
train = pd.read_csv(os.path.join(cst.data_root, "clicks_train.csv"))  # display_id,ad_id,clicked
test = pd.read_csv(os.path.join(cst.data_root, "clicks_test.csv"))  # display_id,ad_id,clicked

doc_cate = pd.read_csv(
        os.path.join(cst.data_root, "documents_categories.csv"))  # document_id,category_id,confidence_level
doc_entity = pd.read_csv(
        os.path.join(cst.data_root, "documents_entities.csv"))  # document_id,entity_id,confidence_level
doc_topics = pd.read_csv(os.path.join(cst.data_root, "documents_topics.csv"))  # document_id,topic_id,confidence_level
events = pd.read_csv(os.path.join(cst.data_root, "events.csv"))  # document_id,topic_id,confidence_level
promoted_content = pd.read_csv(
        os.path.join(cst.data_root, "promoted_content.csv"))  # ad_id,document_id,campaign_id,advertiser_id

train.head()

doc_cate.rename(columns = {'confidence_level':"doc_cate_conf_level"},inplace=True)
doc_entity.rename(columns = {'confidence_level':"doc_entity_conf_level"},inplace=True)
doc_topics.rename(columns = {'confidence_level':"doc_topics_conf_level"},inplace=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#cal_click_rate
cnt = train[train.clicked == 1].ad_id.value_counts()
cntall = train.ad_id.value_counts()


def get_ad_click_rate(ad_id):
    
    return cnt.get(ad_id,0) / (cntall.get(ad_id,0) + reg)

promoted_content['ad_click_rate'] = promoted_content.ad_id.apply(lambda x:get_ad_click_rate(x))

In [5]:
#join doc info

def mult_merge(l, key):
    x1 = l[0]
    for x in l[1:]:
        x1 = x1.merge(x, on=key,how='outer')
    return x1
doc_info = mult_merge([doc_cate,doc_entity,doc_topics],"document_id")

In [ ]:
#join ad
ad_doc_info = promoted_content.merge(doc_info,on = "document_id")
ad_doc_info.head()

,ad_id,document_id,campaign_id,advertiser_id,ad_click_rate,category_id,doc_cate_conf_level,entity_id,doc_entity_conf_level,topic_id,doc_topics_conf_level
0,1,6614,1,7,0,1209,0.925264,NaN,NaN,NaN,NaN
1,1,6614,1,7,0,1708,0.070466,NaN,NaN,NaN,NaN
2,2,471467,2,7,0,1505,0.920000,NaN,NaN,89,0.122998
3,2,471467,2,7,0,1505,0.920000,NaN,NaN,26,0.081036
4,2,471467,2,7,0,1505,0.920000,NaN,NaN,143,0.066163


In [ ]:
#test.join
rs = test.merge(ad_doc_info,on = "ad_id").merge(events,on="display_id")
rs.to_csv("/Users/dongjian/data/outbrain_featrues")